In [ ]:
from ROOT import gROOT, TFile, TCanvas, TF1, kGreen, kBlue
from ipywidgets import interact, interact_manual, interactive, fixed
from IPython.display import clear_output, display
import ipywidgets as widgets

In [ ]:
file = TFile('data/InvariantMass.root')

In [ ]:
part_type = widgets.Dropdown(options=['K0', 'Lambda', 'AntiLambda'], value='K0', description='Particle type:')
display(part_type)

collision = widgets.Dropdown(options=['pp', 'pbpb'], value='pbpb', description='Collision type:')
display(collision)

centrality = widgets.Dropdown(options=['000_010', '010_020', '020_030', '030_040', '040_050', '050_060', '060_070', '070_080', '080_090', '090_100'], value='000_010', description='Centrality:')
display(centrality)

In [ ]:
directory = file.Get(part_type.value)
histogram_name = f'{collision.value}{"_" + centrality.value if collision.value == "pbpb" else ""}_{part_type.value.lower()}'
print(f'Loading Histogram "{histogram_name}"')
histogram = directory.Get(histogram_name)

In [ ]:
c1 = TCanvas('c1', '', 200, 10, 700, 500)
histogram.Draw()
c1.Draw()

In [ ]:
def fit_histogram(hist, bg, sg):
    bg_min, bg_max = bg
    sg_min, sg_max = sg
    
    signal_fit = TF1('fit', 'gausn(0)+pol2(3)', 0, 2)
    signal_fit.SetParNames('Y', '#mu', '#sigma', 'A', 'B', 'C')
    signal_fit.SetRange(bg_min, bg_max);
    signal_fit.SetParameter('Y', 80)
    signal_fit.SetParameter('#mu', (sg_min + sg_max) / 2)
    signal_fit.SetParameter('#sigma', (sg_max - sg_min) / 4)
    signal_fit.SetParLimits(0, 0, 1e9)
    signal_fit.SetParLimits(1, sg_min, sg_max)
    signal_fit.SetParLimits(2, 0, (sg_max - sg_min) / 2)
    signal_fit.SetLineColor(kGreen + 1)
    
    result = hist.Fit(signal_fit, 'NQSR', '', bg_min, bg_max)
    
    background_fit = TF1('bg', 'pol2', 0, 10)
    background_fit.SetParNames('A', 'B', 'C')
    background_fit.SetRange(bg_min, bg_max);
    background_fit.SetParameter('A', signal_fit.GetParameter('A'))
    background_fit.SetParameter('B', signal_fit.GetParameter('B'))
    background_fit.SetParameter('C', signal_fit.GetParameter('C'))
    background_fit.SetParError(0, signal_fit.GetParError(3))
    background_fit.SetParError(1, signal_fit.GetParError(4))
    background_fit.SetParError(2, signal_fit.GetParError(5))
    background_fit.SetLineColor(kBlue + 1)
    
    return signal_fit, background_fit

wig1 = fixed(histogram)
wig2 = widgets.FloatRangeSlider(value=(0.4,0.6), min=0, max=2, step=0.0001)
wig3 = widgets.FloatRangeSlider(value=(0.48,0.51), min=0, max=2, step=0.0001)

c2 = TCanvas('c2', '', 200, 10, 700, 500)

@interact_manual(hist=wig1, bg=wig2, sg=wig3)
def fit_interactive(hist, bg, sg):
    sig_fit, bg_fit = fit_histogram(hist, bg, sg)
    hist.Draw()
    sig_fit.Draw('same')
    bg_fit.Draw('same')
    c2.Draw()